In [1]:
!pip install pandas 


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import sqlite3
import pandas as pd
from collections import defaultdict

In [3]:
database = "../dataset/database/retro.db"

In [4]:
con: sqlite3.Connection = sqlite3.connect(database)
cur: sqlite3.Cursor = con.cursor()

In [5]:
# 1. How many incidents there are for each bug category
query = """
SELECT vuln_title, VulnerabilitiesMap.category, COUNT(DISTINCT I.incident_id) as total, GROUP_CONCAT(I.incident_title) as ids
FROM VulnerabilitiesMap
LEFT JOIN Cause C on VulnerabilitiesMap.vuln_map_id = C.vuln_map_id
LEFT JOIN IncidentCause IC on C.cause_id = IC.cause_id
LEFT JOIN Incident I on I.incident_id = IC.incident_id
GROUP BY vuln_title
ORDER BY total DESC
"""
incidents_per_category = pd.read_sql(query, con)
incidents_per_category['ids'] = incidents_per_category['ids'].apply(lambda x: list(set([x for x in x.split(',')] if x is not None else [])))
incidents_per_category

,vuln_title,category,total,ids
0,Other auxiliary vulnerabilities,Auxilary,48,"[20210627_SharedStake_S, 20190928_Fusion_S, 20..."
1,Absence of coding logic or sanity check,Smart Contract,42,"[20220312_Paraluni_S, 20210810_PunkProtocol_CS..."
2,On-chain oracle manipulation,Protocol,29,"[20210603_PancakeHunny_S, 20211006_MyFarmPet_S..."
3,Reentrancy,Smart Contract,13,"[20220312_Paraluni_S, 2020117_OriginDollar_CSR..."
4,Backdoor/Honeypot,Auxilary,12,"[20210304_MeerkatFinance_CSR, 20201201_Compoun..."
5,"Liquidity borrow, purchase, mint, deposit",Protocol,10,"[2020117_OriginDollar_CSR, 20211123_PloutozFin..."
6,Camouflage a token contract,Protocol,9,"[20220312_Paraluni_S, 20201011_WLEO_S, 2021031..."
7,Token standard incompatibility,Protocol,8,"[20220425_WienerDoge_S, 20220327_RevestFinance..."
8,Function/State Visibility Error,Smart Contract,8,"[20200630_VETH_S, 20200618_Bancor_S, 20210527_..."
9,Other unsafe DeFi protocol dependency,Protocol,7,"[20210508_RariCapital_CSR, 20210221_PrimitiveF..."


In [6]:
# 2. Only the categories we include in the study
incidents_per_ic_category = incidents_per_category.loc[incidents_per_category['category'].isin(['Smart Contract', 'Protocol'])]
incidents_per_ic_category


,vuln_title,category,total,ids
1,Absence of coding logic or sanity check,Smart Contract,42,"[20220312_Paraluni_S, 20210810_PunkProtocol_CS..."
2,On-chain oracle manipulation,Protocol,29,"[20210603_PancakeHunny_S, 20211006_MyFarmPet_S..."
3,Reentrancy,Smart Contract,13,"[20220312_Paraluni_S, 2020117_OriginDollar_CSR..."
5,"Liquidity borrow, purchase, mint, deposit",Protocol,10,"[2020117_OriginDollar_CSR, 20211123_PloutozFin..."
6,Camouflage a token contract,Protocol,9,"[20220312_Paraluni_S, 20201011_WLEO_S, 2021031..."
7,Token standard incompatibility,Protocol,8,"[20220425_WienerDoge_S, 20220327_RevestFinance..."
8,Function/State Visibility Error,Smart Contract,8,"[20200630_VETH_S, 20200618_Bancor_S, 20210527_..."
9,Other unsafe DeFi protocol dependency,Protocol,7,"[20210508_RariCapital_CSR, 20210221_PrimitiveF..."
11,"Other inconsistent, improper or unprotected ac...",Smart Contract,5,"[20210318_SILFinance_S, 20220201_MOX_S, 202109..."
13,Logic Errors,Smart Contract,5,"[20201228_CoverProtocol_CSR, 20210507_ValueDeF..."


In [7]:
# 3. Count how many incidents are in scope
# Protocols which have at least one Smart Contract or Protocol vulnerability
# Note that they might also have non-smart contract or protocol related causes
query = """
SELECT DISTINCT incident_title
FROM Incident
JOIN IncidentCause IC on Incident.incident_id = IC.incident_id
JOIN Cause C on C.cause_id = IC.cause_id
JOIN VulnerabilitiesMap VM on C.vuln_map_id = VM.vuln_map_id
WHERE VM.category in ('Smart Contract', 'Protocol')
"""
incidents_in_scope = pd.read_sql(query, con)
incidents = list(incidents_in_scope.incident_title)
incidents_in_scope

,incident_title
0,20210208_BTFinance_CS
1,20210629_MerlinLab_RS
2,20200419_LendfMe_S
3,20211003_Compound_SR
4,20200913_bzx_CS
...,...
122,20210718_ArrayFinance_S
123,20210127_refi
124,20220415_RikkeiFinance_S
125,20200929_Eminence_CSR


In [8]:
### 4. For each Incident that we have in our dataset count how many contracts where attacked
query = """
SELECT incident_title, COUNT(VC.vulnerable_contract_id)
FROM Incident
JOIN Victim V on Incident.incident_id = V.incident_id
JOIN VulnerableContract VC on V.victim_id = VC.victim_id
WHERE incident_title in ({})
GROUP BY incident_title
""".format(','.join('"' + i + '"' for i in incidents))
vuln_contracts_per_incident = pd.read_sql(query, con)
vuln_contracts_per_incident

,incident_title,COUNT(VC.vulnerable_contract_id)
0,20200215_bZx_CS,1
1,20200218_bZx_S,1
2,20200418_Uniswap_S,1
3,20200419_LendfMe_S,1
4,20200618_Bancor_S,3
...,...,...
114,20220425_Medamon_S,2
115,20220425_PI-DAO_S,2
116,20220425_WienerDoge_S,2
117,20220430_FeiProtocol_CSR,1


In [9]:
# 5. Get incidents for which we have at least one contract with source code
query = """
SELECT incident_title, COUNT(vulnerable_contract_id)
FROM Incident
JOIN Victim V on Incident.incident_id = V.incident_id
JOIN VulnerableContract VC on V.victim_id = VC.victim_id
WHERE incident_title in ({})
AND VC.has_source = 1
GROUP BY incident_title
""".format(','.join('"' + i + '"' for i in incidents))
incidents_with_source = pd.read_sql(query, con)
incidents_with_source

,incident_title,COUNT(vulnerable_contract_id)
0,20200215_bZx_CS,1
1,20200218_bZx_S,1
2,20200418_Uniswap_S,1
3,20200419_LendfMe_S,1
4,20200618_Bancor_S,3
...,...,...
97,20220421_ZEED_S,1
98,20220425_Medamon_S,2
99,20220425_PI-DAO_S,1
100,20220425_WienerDoge_S,2


In [10]:
# 6. For each incident check how many analyses what was the success rate of the analyses
query = """
SELECT incident_title,
       SUM(CASE WHEN error = 'True' THEN 1 ELSE 0 ENd) Errors,
       SUM(CASE WHEN error = 'False' THEN 1 ELSE 0 ENd) Success
FROM Incident
JOIN Victim V on Incident.incident_id = V.incident_id
JOIN VulnerableContract VC on V.victim_id = VC.victim_id
JOIN RetroAnalysis RA on VC.vulnerable_contract_id = RA.vulnerable_contract_id
WHERE incident_title in ({})
GROUP BY incident_title
""".format(','.join('"' + i + '"' for i in incidents))
analyses_per_incident = pd.read_sql(query, con)
analyses_per_incident["Success Rate"] = analyses_per_incident["Success"] / (analyses_per_incident["Success"] + analyses_per_incident["Errors"])
analyses_per_incident.sort_values(by=['Success Rate'])

,incident_title,Errors,Success,Success Rate
114,20220722_THORChain_CSR,3,4,0.571429
55,20210629_THORChain_CS,3,4,0.571429
56,20210710_Anyswap_SR,3,4,0.571429
62,20210730_Levyathan_CSR,3,4,0.571429
64,20210804_WaultFinance_S,3,4,0.571429
...,...,...,...,...
90,20211211_SorbetFinance_S,0,2,1.000000
35,20210501_Spartan_CSR,0,7,1.000000
88,20211127_dydx_S,0,2,1.000000
67,20210812_NekoNetwork_S,0,2,1.000000


In [11]:
# 7. For each tool count its success rate
query = """
SELECT tool, is_bytecode,
       SUM(CASE WHEN error = 'True' THEN 1 ELSE 0 ENd) Errors,
       SUM(CASE WHEN error = 'False' THEN 1 ELSE 0 ENd) Success
FROM RetroAnalysis
GROUP BY tool, is_bytecode
"""
success_rate_per_tool = pd.read_sql(query, con)
success_rate_per_tool["Success Rate"] = success_rate_per_tool["Success"] / (success_rate_per_tool["Success"] + success_rate_per_tool["Errors"])
success_rate_per_tool.sort_values(by=['Success Rate'])

,tool,is_bytecode,Errors,Success,Success Rate
0,confuzzius,False,128,26,0.168831
1,mythril,False,21,133,0.863636
3,oyente,False,21,133,0.863636
5,slither,False,2,152,0.987013
2,mythril,True,0,172,1.000000
4,oyente,True,0,172,1.000000
6,solhint,False,0,154,1.000000


In [12]:
# 8. For each tool check if there is any findings for the specific bug
# Get incidents and vulnerabilities of each incident
query = """
SELECT incident_title, vuln_title
FROM Incident
JOIN IncidentCause IC on Incident.incident_id = IC.incident_id
JOIN Cause C on C.cause_id = IC.cause_id
JOIN VulnerabilitiesMap VM on C.vuln_map_id = VM.vuln_map_id
WHERE VM.category in ('Smart Contract', 'Protocol')
"""
df_incidents_vulnerabilities = pd.read_sql(query, con)
qset = cur.execute(query)
incidents_vulnerabilities = defaultdict(set)
# Get for incident 
for incident_title, vuln_title in qset:
    incidents_vulnerabilities[incident_title].add(vuln_title)
# If we do Left join to SCV and VM we'll also get vuln_title = null which represents warnings
query = """
SELECT incident_title, contract_address, tool, is_bytecode, vuln_title, occurences
FROM Incident
JOIN Victim V on Incident.incident_id = V.incident_id
JOIN VulnerableContract VC on V.victim_id = VC.victim_id
JOIN RetroAnalysis RA on VC.vulnerable_contract_id = RA.vulnerable_contract_id
JOIN RetroFindings RF on RA.analysis_id = RF.analysis_id
JOIN SmartContractVulnerabilities SCV on RF.vulnerability_type_id = SCV.vulnerability_type_id
JOIN VulnerabilitiesMap VM on SCV.vuln_map_id = VM.vuln_map_id
WHERE incident_title in ({})
GROUP BY incident_title, contract_address, tool, is_bytecode, vuln_title, occurences
""".format(','.join('"' + i + '"' for i in incidents))
print(query)


SELECT incident_title, contract_address, tool, is_bytecode, vuln_title, occurences
FROM Incident
JOIN Victim V on Incident.incident_id = V.incident_id
JOIN VulnerableContract VC on V.victim_id = VC.victim_id
JOIN RetroAnalysis RA on VC.vulnerable_contract_id = RA.vulnerable_contract_id
JOIN RetroFindings RF on RA.analysis_id = RF.analysis_id
JOIN SmartContractVulnerabilities SCV on RF.vulnerability_type_id = SCV.vulnerability_type_id
JOIN VulnerabilitiesMap VM on SCV.vuln_map_id = VM.vuln_map_id
WHERE incident_title in ("20210208_BTFinance_CS","20210629_MerlinLab_RS","20200419_LendfMe_S","20211003_Compound_SR","20200913_bzx_CS","20220320_Lifinance_S","20200218_bZx_S","20210622_ElevenFinance_CSR","20210810_PunkProtocol_CSR","20210816_XSURGE_S","20210603_PancakeHunny_S","20210730_Levyathan_CSR","20220402_InverseFinance_CR","20211123_PloutozFinance_S","20200913_SoftYearn_CS","20201026_HarvestFinance_CSR","20200418_Uniswap_S","20210522_BoggedFinance_S","20220722_THORChain_CSR","20201117_8

In [13]:
df_incidents_findings = pd.read_sql(query, con)
qset = cur.execute(query)
incidents_findings = defaultdict(set)
for incident_title, _, _, _, vuln_title, _ in qset:
    incidents_findings[incident_title].add(vuln_title)

# incident_title, total_vulns, total_detected_vulns
incidents_aggr_results = []
for incident, vulns in incidents_vulnerabilities.items():
    if incident not in incidents_findings:
        detected = 0
    else:
        detected = sum(1 for vuln in vulns if vuln in incidents_findings[incident])
    incidents_aggr_results.append([incident, len(vulns), detected])
df_incidents_aggr_results = pd.DataFrame(
    incidents_aggr_results, columns = ["Incident", "Total Vulns", "Detected Vulns"]
)

df_incidents_aggr_results["At least one"] = df_incidents_aggr_results["Detected Vulns"] > 0
df_incidents_aggr_results["All"] = df_incidents_aggr_results["Total Vulns"] == df_incidents_aggr_results["Detected Vulns"]
print("Total Incidents: {}, At least one: {}, All {}".format(
    len(df_incidents_aggr_results.index),
    len(df_incidents_aggr_results[df_incidents_aggr_results['At least one'] == True]),
    len(df_incidents_aggr_results[df_incidents_aggr_results['All'] == True]),
))

Total Incidents: 127, At least one: 16, All 6


In [14]:
# 9. For each category find how many incidents could have been prevented and how many were reported
prev_incidents_per_category = incidents_per_ic_category

vulnerabilities_detected = {t: set() for t in incidents_per_ic_category.vuln_title}
vulnerabilities_findings = {t: set() for t in incidents_per_ic_category.vuln_title}
for incident, vulns in incidents_vulnerabilities.items():
    for vuln in vulns:
        if vuln in incidents_findings[incident]:
            vulnerabilities_detected[vuln].add(incident)
    for vuln in incidents_findings[incident]:
        if vuln in vulnerabilities_findings:
            vulnerabilities_findings[vuln].add(incident)
prev_incidents_per_category['Detectable'] = list(len(i) for i in vulnerabilities_detected.values())
prev_incidents_per_category['detectable_ids'] = list(vulnerabilities_detected.values())
prev_incidents_per_category['Findings'] = list(len(i) for i in vulnerabilities_findings.values())
prev_incidents_per_category['detectable_ids'] = list(vulnerabilities_detected.values())
prev_incidents_per_category['findings_ids'] = list(vulnerabilities_findings.values())
prev_incidents_per_category

/var/folders/fq/m8lfjxgn0973ymrn76p0x_cw0000gn/T/ipykernel_1436/544461778.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_incidents_per_category['Detectable'] = list(len(i) for i in vulnerabilities_detected.values())
/var/folders/fq/m8lfjxgn0973ymrn76p0x_cw0000gn/T/ipykernel_1436/544461778.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_incidents_per_category['detectable_ids'] = list(vulnerabilities_detected.values())
/var/folders/fq/m8lfjxgn0973ymrn76p0x_cw0000gn/T/ipykernel_1436/544461

,vuln_title,category,total,ids,Detectable,detectable_ids,Findings,findings_ids
1,Absence of coding logic or sanity check,Smart Contract,42,"[20220312_Paraluni_S, 20210810_PunkProtocol_CS...",0,{},0,{}
2,On-chain oracle manipulation,Protocol,29,"[20210603_PancakeHunny_S, 20211006_MyFarmPet_S...",0,{},0,{}
3,Reentrancy,Smart Contract,13,"[20220312_Paraluni_S, 2020117_OriginDollar_CSR...",11,"{20220312_Paraluni_S, 20220327_RevestFinance_S...",82,"{20210627_SharedStake_S, 20211006_MyFarmPet_S,..."
5,"Liquidity borrow, purchase, mint, deposit",Protocol,10,"[2020117_OriginDollar_CSR, 20211123_PloutozFin...",0,{},0,{}
6,Camouflage a token contract,Protocol,9,"[20220312_Paraluni_S, 20201011_WLEO_S, 2021031...",0,{},0,{}
7,Token standard incompatibility,Protocol,8,"[20220425_WienerDoge_S, 20220327_RevestFinance...",0,{},0,{}
8,Function/State Visibility Error,Smart Contract,8,"[20200630_VETH_S, 20200618_Bancor_S, 20210527_...",5,"{20200630_VETH_S, 20200618_Bancor_S, 20210527_...",67,"{20220312_Paraluni_S, 20210603_PancakeHunny_S,..."
9,Other unsafe DeFi protocol dependency,Protocol,7,"[20210508_RariCapital_CSR, 20210221_PrimitiveF...",0,{},0,{}
11,"Other inconsistent, improper or unprotected ac...",Smart Contract,5,"[20210318_SILFinance_S, 20220201_MOX_S, 202109...",0,{},19,"{20201228_CoverProtocol_CSR, 20201128_SushiSwa..."
13,Logic Errors,Smart Contract,5,"[20201228_CoverProtocol_CSR, 20210507_ValueDeF...",0,{},0,{}


In [15]:
# 10. Tools
query = """
SELECT vm.vuln_title, t.tool_name
FROM ToolsVulnerabilities tv
JOIN Tool t ON tv.tool_id = t.tool_id
JOIN VulnerabilitiesMap VM on tv.vuln_map_id = VM.vuln_map_id;
"""
qset =  cur.execute(query)
tools_df = incidents_per_ic_category
tools = {v: [] for v in incidents_per_ic_category.vuln_title}
for vuln, tool in qset:
    tools[vuln].append(tool)
tools_df['Tools'] = list(tools.values())
tools_df

/var/folders/fq/m8lfjxgn0973ymrn76p0x_cw0000gn/T/ipykernel_1436/4038342048.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tools_df['Tools'] = list(tools.values())


,vuln_title,category,total,ids,Detectable,detectable_ids,Findings,findings_ids,Tools
1,Absence of coding logic or sanity check,Smart Contract,42,"[20220312_Paraluni_S, 20210810_PunkProtocol_CS...",0,{},0,{},[]
2,On-chain oracle manipulation,Protocol,29,"[20210603_PancakeHunny_S, 20211006_MyFarmPet_S...",0,{},0,{},[]
3,Reentrancy,Smart Contract,13,"[20220312_Paraluni_S, 2020117_OriginDollar_CSR...",11,"{20220312_Paraluni_S, 20220327_RevestFinance_S...",82,"{20210627_SharedStake_S, 20211006_MyFarmPet_S,...","[slither, oyente, confuzzius, mythril, solhint]"
5,"Liquidity borrow, purchase, mint, deposit",Protocol,10,"[2020117_OriginDollar_CSR, 20211123_PloutozFin...",0,{},0,{},[]
6,Camouflage a token contract,Protocol,9,"[20220312_Paraluni_S, 20201011_WLEO_S, 2021031...",0,{},0,{},[]
7,Token standard incompatibility,Protocol,8,"[20220425_WienerDoge_S, 20220327_RevestFinance...",0,{},0,{},[]
8,Function/State Visibility Error,Smart Contract,8,"[20200630_VETH_S, 20200618_Bancor_S, 20210527_...",5,"{20200630_VETH_S, 20200618_Bancor_S, 20210527_...",67,"{20220312_Paraluni_S, 20210603_PancakeHunny_S,...",[solhint]
9,Other unsafe DeFi protocol dependency,Protocol,7,"[20210508_RariCapital_CSR, 20210221_PrimitiveF...",0,{},0,{},[]
11,"Other inconsistent, improper or unprotected ac...",Smart Contract,5,"[20210318_SILFinance_S, 20220201_MOX_S, 202109...",0,{},19,"{20201228_CoverProtocol_CSR, 20201128_SushiSwa...","[slither, oyente, confuzzius, mythril, solhint]"
13,Logic Errors,Smart Contract,5,"[20201228_CoverProtocol_CSR, 20210507_ValueDeF...",0,{},0,{},[]


In [ ]:
# For each incident get a list with vulns
query = """
SELECT DISTINCT
    I.incident_title,
    VM.vuln_title
FROM Incident I
JOIN IncidentCause IC ON I.incident_id = IC.incident_id
JOIN Cause C ON IC.cause_id = C.cause_id
JOIN VulnerabilitiesMap VM ON C.vuln_map_id = VM.vuln_map_id;
"""
qset = cur.execute(query)
incident_vulns = defaultdict(list)
for incident, vuln in qset:
    incident_vulns[incident].append(vuln)

incident_vulns
    


In [16]:
# 10. Percentages
# Damage per incident
query = """
SELECT incident_title, avg_reported_damage_in_usd as damage
FROM Incident
"""
qset = cur.execute(query)
damage = {title: damage for title, damage in qset}

total_vuln_categories = tools_df['vuln_title'].count()

incidents = df_incidents_aggr_results['Incident']
in_scope_incidents = set(incidents)
total_incidents = df_incidents_aggr_results['Incident'].count()
total_incidents_in_scope = tools_df[tools_df['Tools'].apply(len) > 0]['ids']
incidents_in_scope = set(i for l in total_incidents_in_scope for i in l)
total_incidents_in_scope = len(incidents_in_scope)
incidents_out_of_scope = set(i for i in incidents if i not in incidents_in_scope)
total_incidents_out_of_scope = len(incidents_out_of_scope)
detectable_incidents = set(i for l in tools_df["detectable_ids"] for i in l)
total_detectable_incidents = len(detectable_incidents)

total_value = int(sum(damage[i] for i in df_incidents_aggr_results['Incident']))
total_value_out_of_scope = int(sum(damage[i] for i in incidents_out_of_scope))
total_value_in_scope = int(sum(damage[i] for i in incidents_in_scope))
total_value_could_have_been_saved = int(sum(damage[i] for i in detectable_incidents))

# In scope of selected tools
print(f"Total Incidents:                        {total_incidents}")
print(f"Total Damage:                           {total_value}")
print(f"Total Incidents out of scope:           {total_incidents_out_of_scope} ({int(total_incidents_out_of_scope/total_incidents * 100)}%)")
print(f"Total Damage out of scope:              {total_value_out_of_scope} ({int(total_value_out_of_scope/total_value * 100)}%)")
print(f"Total Incidents in scope:               {total_incidents_in_scope} ({int(total_incidents_in_scope/total_incidents * 100)}%)")
print(f"Total Damage in scope:                  {total_value_in_scope} ({int(total_value_in_scope/total_value * 100)}%)")
print(f"Total Detectable Incidents in scope:    {total_detectable_incidents} ({int(total_detectable_incidents/total_incidents_in_scope * 100)}%)")
print(f"Total Damage could have been prevented: {total_value_could_have_been_saved} ({int(total_value_could_have_been_saved/total_value_in_scope * 100)}%)")

exploits_with_logic = {
    i for i in damage
    if 'Absence of coding logic or sanity check' in incident_vulns[i] or 'Logic Errors' in incident_vulns[i]
}
logic_damage = int(sum(d for i, d in damage.items() if i in exploits_with_logic))
print(f"Total Logic:                            {len(exploits_with_logic)} ({int(len(exploits_with_logic)/total_incidents * 100)}%)")
print(f"Total Logic Damage:                     {logic_damage} ({int(logic_damage/total_value * 100)}%)")
merged_incidents_in_scope = incidents_in_scope.union(exploits_with_logic)
merged_incidents_in_scope_damage = int(sum(d for i, d in damage.items() if i in merged_incidents_in_scope))
print(f"Total Logic + In-scope:                 {len(merged_incidents_in_scope)} ({int(len(merged_incidents_in_scope)/total_incidents * 100)}%)")
print(f"Total Logic + In-scope Damage:          {merged_incidents_in_scope_damage} ({int(merged_incidents_in_scope_damage/total_value * 100)}%)")
#df_incidents_damage.loc[df_incidents_damage['incident_title'] == '20210522_DeFi100_S']['damage'].values[0]


#set(i for l in tools_df[tools_df['Tools'].apply(len) == 0]['ids'])

Total Incidents:                        127
Total Damage:                           2331903028
Total Incidents out of scope:           95 (74%)
Total Damage out of scope:              2060349987 (88%)
Total Incidents in scope:               32 (25%)
Total Damage in scope:                  271553041 (11%)
Total Detectable Incidents in scope:    16 (50%)
Total Damage could have been prevented: 160596783 (59%)


NameError: name 'incident_vulns' is not defined

In [ ]:
###### Get tools and analysis mode
query = """
SELECT DISTINCT tool, is_bytecode
FROM RetroAnalysis
"""
tools_analysis_df = pd.read_sql(query, con)
total_index = len(tools_analysis_df.index)
tools_analysis_df.loc[total_index] = ['total', '']
tools_analysis_df




In [ ]:
# For each tool get a list with vulns that are in scope
query = """
SELECT
    T.tool_name,
    VM.vuln_title
FROM Tool T
JOIN ToolsVulnerabilities TV ON T.tool_id = TV.tool_id
JOIN VulnerabilitiesMap VM ON TV.vuln_map_id = VM.vuln_map_id;
"""
qset = cur.execute(query)
tool_vulns = defaultdict(list)
for tool, vuln in qset:
    tool_vulns[tool].append(vuln)
    
# Find incidents that all contracts have source
query = """
SELECT
    DISTINCT I.incident_title
FROM
    Incident I
WHERE
    I.incident_id NOT IN (
        SELECT
            DISTINCT V.incident_id
        FROM
            Victim V
        JOIN VulnerableContract VC ON V.victim_id = VC.victim_id
        WHERE
            VC.has_source = 0
    );
"""
qset = cur.execute(query)
incidents_with_source = set([i[0] for i in qset])

# Find findings per tool for incidents
query = """
SELECT
    I.incident_title,
    RA.tool,
    RA.is_bytecode,
    VM.vuln_title,
    VC.contract_address
FROM
    Incident I
JOIN
    Victim V ON I.incident_id = V.incident_id
JOIN
    VulnerableContract VC ON V.victim_id = VC.victim_id
JOIN
    RetroAnalysis RA ON VC.vulnerable_contract_id = RA.vulnerable_contract_id
JOIN
    RetroFindings RF ON RA.analysis_id = RF.analysis_id
JOIN
    SmartContractVulnerabilities SCV ON RF.vulnerability_type_id = SCV.vulnerability_type_id
JOIN
    VulnerabilitiesMap VM ON SCV.vuln_map_id = VM.vuln_map_id;
"""
qset = cur.execute(query)
findings = defaultdict(lambda: defaultdict(list))
findings_merged = defaultdict(lambda: defaultdict(set))
findings_contracts = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
for incident, tool, is_bytecode, vuln_title, contract_address in qset:
    findings[(tool, is_bytecode)][incident].append(vuln_title)
    findings_merged[tool][incident].add(vuln_title)
    findings_contracts[(tool, is_bytecode)][incident][vuln_title].append(contract_address)

# Create a function to check if there's an intersection between two lists
def has_intersection(list1, list2):
    return len(set(list1) & set(list2)) > 0

###### Find incidents in scope
# Add a new column 'incidents_in_scope' to tools_analysis_df
tools_analysis_df['incidents_in_scope'] = ""

tools_analysis_df.at[total_index, 'incidents_in_scope'] = set()
# Iterate over the rows in tools_analysis_df
for index, row in tools_analysis_df.iterrows():
    if index == total_index:
        continue
    tool = row['tool']
    # Get the tool's vulnerabilities
    tool_vulnerabilities = tool_vulns[tool]
    # Get the incident titles that have at least one vulnerability in the tool's vulnerabilities
    incidents_in_scope = set([
        incident for incident, vulns in incident_vulns.items() 
        if has_intersection(vulns, tool_vulnerabilities)
    ])
    # Update the 'incidents_in_scope' column
    tools_analysis_df.at[index, 'incidents_in_scope'] = incidents_in_scope
    tools_analysis_df.at[total_index, 'incidents_in_scope'].update(incidents_in_scope)

    
print(tools_analysis_df)

In [ ]:
##### Incidents that could have been prevented
tools_analysis_df['prevented_incidents'] = ""
tools_analysis_df['false_positives'] = ""

tools_analysis_df.at[total_index, 'prevented_incidents'] = set()
tools_analysis_df.at[total_index, 'false_positives'] = set()
#Iterate over the rows in tools_analysis_df
for index, row in tools_analysis_df.iterrows():
    if index == total_index:
        continue
    tool = row['tool']
    is_bytecode = row['is_bytecode']
    # Get the incident titles that have at least one vulnerability in the tool's vulnerabilities
    incidents_in_scope = row['incidents_in_scope']
    prevented_incidents = set([
        incident for incident in incidents_in_scope
        if has_intersection(incident_vulns[incident], findings[(tool, is_bytecode)][incident])
    ])
    # We manually check the solhint bugs and all of them are FP so we should remove them.
    false_positives = set()
    if tool == 'solhint':
        false_positives = prevented_incidents
        prevented_incidents = []
    # Update the 'prevented_incidents' column
    tools_analysis_df.at[index, 'prevented_incidents'] = prevented_incidents
    tools_analysis_df.at[index, 'false_positives'] = false_positives
    tools_analysis_df.at[total_index, 'prevented_incidents'].update(prevented_incidents)
    tools_analysis_df.at[total_index, 'false_positives'].update(false_positives)

In [ ]:
##### Merge tools (is_bytecode)
df_new = tools_analysis_df.copy()[0:0]
for index, row in tools_analysis_df.iterrows():
    if row['tool'] not in set(df_new['tool']):
        df_new.loc[len(df_new)] = row
    else:
        loc = df_new.loc[df_new['tool'] == row['tool']].index[0]
        df_new.at[loc, 'prevented_incidents'].update(row['prevented_incidents'])
        df_new.at[loc, 'false_positives'].update(row['false_positives'])
        df_new.at[loc, 'incidents_in_scope'].update(row['incidents_in_scope'])
tools_analysis_df = df_new
tools_analysis_df = tools_analysis_df.drop(columns=['is_bytecode'])

In [ ]:
# Damage per incident
query = """
SELECT incident_title, avg_reported_damage_in_usd as damage
FROM Incident
"""
qset = cur.execute(query)
damage = {title: damage for title, damage in qset}
    
##### Damage in scope per tool
tools_analysis_df['damage_in_scope'] = ""

#Iterate over the rows in tools_analysis_df
for index, row in tools_analysis_df.iterrows():
    tool = row['tool']
    # Get the incident titles that have at least one vulnerability in the tool's vulnerabilities
    incidents_in_scope = row['incidents_in_scope']
    damage_in_scope = int(sum(
        damage[incident] for incident in incidents_in_scope
    ))
    # Update the 'incidents_in_scope' column
    tools_analysis_df.at[index, 'damage_in_scope'] = damage_in_scope

    

In [ ]:
##### Damage in scope per tool
tools_analysis_df['prevented_damage'] = ""

#Iterate over the rows in tools_analysis_df
for index, row in tools_analysis_df.iterrows():
    tool = row['tool']
    # Get the incident titles that have at least one vulnerability in the tool's vulnerabilities
    prevented_incidents = row['prevented_incidents']
    prevented_damage = int(sum(
        damage[incident] for incident in prevented_incidents
    ))
    # Update the 'incidents_in_scope' column
    tools_analysis_df.at[index, 'prevented_damage'] = prevented_damage

In [ ]:
##### print #####
def list_length(df):
    # Create a new DataFrame with the same structure as the input
    new_df = df.copy()

    # Iterate over columns in the input DataFrame
    for column in df.columns:
        # Check if the first element in the column is a list
        if isinstance(df[column].iloc[0], set):
            # Replace each list with its length
            new_df[column] = df[column].apply(len)
        else:
            # Keep the original value if it's not a list
            new_df[column] = df[column]

    return new_df


df_11_to_print = list_length(tools_analysis_df)
df_11_to_print

In [ ]:
# 12.
query = """
SELECT DISTINCT tool_name
FROM Tool;
"""
qset =  cur.execute(query)
vuln_analysis_tool = pd.DataFrame()

vuln_analysis_tool['vuln'] = ""
for tool in qset:
    if tool[0] == 'manticore':
        continue
    vuln_analysis_tool[tool[0] + "_detected"] = ""
    vuln_analysis_tool[tool[0] + "_not_exploited"] = ""
vuln_analysis_tool['total_attacks'] = ""
vuln_analysis_tool['total_detected'] = ""
vuln_analysis_tool['total_not_exploited'] = ""
for index, row in tools_df.iterrows():
    df_12_index = len(vuln_analysis_tool)
    if len(row['Tools']) == 0:
        continue
    vuln_analysis_tool.at[df_12_index, 'vuln'] = row['vuln_title']
    vuln_analysis_tool.at[df_12_index, 'total_attacks'] = row['total']
    detected_ids = set()
    not_exploited_ids = set()
    for tool in row['Tools']:
        tool_detected = {
            incident for incident, findings in findings_merged[tool].items()
            for finding in findings
            if finding == row['vuln_title'] and incident in row['ids']
        }
        vuln_analysis_tool.at[df_12_index, tool + "_detected"] = len(tool_detected)
        detected_ids.update(tool_detected)
        tool_not_exploited = {
            incident for incident, findings in findings_merged[tool].items()
            for finding in set(findings)
            if finding == row['vuln_title'] and incident not in row['ids'] and incident in in_scope_incidents
        }
        vuln_analysis_tool.at[df_12_index, tool + "_not_exploited"] = len(tool_not_exploited)
        not_exploited_ids.update(tool_not_exploited)
    vuln_analysis_tool.at[df_12_index, 'total_detected'] = len(detected_ids)
    vuln_analysis_tool.at[df_12_index, 'total_not_exploited'] = len(not_exploited_ids)

    
def f_format(x):
    if isinstance(x, float):
        return "\\xmark"
    if isinstance(x, int):
        return f"\\num{{{x}}}"
    return f"{x}"
vuln_analysis_tool
# for _, row in vuln_analysis_tool.iterrows():
#     print("{}\\\\".format(" & ".join(f_format(c) for c in row)))

In [ ]:
# Damage per incident
query = """
SELECT incident_title, avg_reported_damage_in_usd as damage
FROM Incident
"""
qset = cur.execute(query)
damage = {title: damage for title, damage in qset}

# For each incident get a list with vulns
query = """
SELECT DISTINCT
    I.incident_title,
    VM.vuln_title
FROM Incident I
JOIN IncidentCause IC ON I.incident_id = IC.incident_id
JOIN Cause C ON IC.cause_id = C.cause_id
JOIN VulnerabilitiesMap VM ON C.vuln_map_id = VM.vuln_map_id;
"""
qset = cur.execute(query)
incident_vulns = defaultdict(list)
for incident, vuln in qset:
    incident_vulns[incident].append(vuln)



In [ ]:
query = """
SELECT DISTINCT tool_name
FROM Tool;
"""
qset =  cur.execute(query)
vuln_analysis_per_tool = pd.DataFrame()

vuln_analysis_per_tool['vuln'] = ""
for tool in qset:
    if tool[0] == 'manticore':
        continue
    vuln_analysis_per_tool[tool[0] + "_detected"] = ""
    vuln_analysis_per_tool[tool[0] + "_not_exploited"] = ""
vuln_analysis_per_tool['total_attacks'] = ""
vuln_analysis_per_tool['total_detected'] = ""
vuln_analysis_per_tool['total_not_exploited'] = ""
for index, row in tools_df.iterrows():
    vuln_analysis_per_tool_index = len(vuln_analysis_per_tool)
    if len(row['Tools']) == 0:
        continue
    vuln_analysis_per_tool.at[vuln_analysis_per_tool_index, 'vuln'] = row['vuln_title']
    vuln_analysis_per_tool.at[vuln_analysis_per_tool_index, 'total_attacks'] = row['total']
    detected_ids = set()
    not_exploited_ids = set()
    for tool in row['Tools']:
        tool_detected = {
            incident for incident, findings in findings_merged[tool].items()
            for finding in findings
            if finding == row['vuln_title'] and incident in row['ids']
        }
        vuln_analysis_per_tool.at[vuln_analysis_per_tool_index, tool + "_detected"] = tool_detected
        detected_ids.update(tool_detected)
        tool_not_exploited = {
            incident for incident, findings in findings_merged[tool].items()
            for finding in set(findings)
            if finding == row['vuln_title'] and incident not in row['ids'] and incident in in_scope_incidents
        }
        vuln_analysis_per_tool.at[vuln_analysis_per_tool_index, tool + "_not_exploited"] = tool_not_exploited
        not_exploited_ids.update(tool_not_exploited)
    vuln_analysis_per_tool.at[vuln_analysis_per_tool_index, 'total_detected'] = detected_ids
    vuln_analysis_per_tool.at[vuln_analysis_per_tool_index, 'total_not_exploited'] = not_exploited_ids
        
    
def f_format(x):
    if isinstance(x, float):
        return "\\xmark"
    if isinstance(x, int):
        return f"\\num{{{x}}}"
    return f"{x}"

vuln_analysis_per_tool